##       VAIC_23_24程序详解
本文对VAIC_23_24程序的YOLOv3神经网络的建立、物体的识别和推理、Intel D435摄像机工作机制、Jetson Nano与Vex-Brian通信过程、远程实时显示的机理、双机协同通信进行运动配合以及利用神经网络识别物体的位置距离信息通过Brian控制机器人运动的技术进行详细的解释说明。

VAIC软件的AI功能是由JetsonExample目录的python程序实现的，C_23_24Jetson端python程序组成已在前面进行简单介绍。请参考“VAIVC软件解释”。作为AI项目，首先对这部分进行技术分析，以让读者尽早理解和把握AI功能在程序中的运用和实现。

###   1.  &nbsp; &nbsp;   VAIC神经网络的建立

Vex AI引进的AI技术是使用深度神经网络对摄像机拍摄的场地实时图像进行物体的识别和距离测量，并将物体的形状、颜色和位置数据送给Vex Brian，由Brian控制机器人。<br>
本节对深度神经网络YOLOv3导入，接收摄像机Intel D435的图像、利用TensorRt高速推理识别物体和距离、将识别物体数据传输至Brian的过程进行深入介绍。<br>



AI过程从创建Class Processing的实例开始。根据python的执行机制，在分配了实例内存空间后，执行processing的__init__(self)函数。<br>

In [ ]:
self.processing = Processing(self.camera.depth_scale)

根据python的执行机制，在分配了实例内存空间后，执行processing的__init__(self)函数，__init__(self)程序段如下。<br>
参数depth_scale是摄像机在深度计算时比例数，camera的初始化时按设定配置从摄像机取得，将在Camera程序部分加于解释。<br>

In [ ]:
    # Class to handle camera data processing, preparing for inference, and running inference 
    # on camera image.
    def __init__(self, depth_scale):
        self.depth_scale = depth_scale
        self.align_to = rs.stream.color
        self.align = rs.align(self.align_to)  # Align depth frames to color stream
        self.model = Model()  # Initialize the object detection model

.self.depth_scale = depth_scale将作为外部参数的深度计算比例存入内部参数，为计算识别物体距离时使用。<br>
.self.align_to = rs.stream.color<br>
.self.align = rs.align(self.align_to)<br>
图像对齐，在RealSense的应用中，深度和物体颜色图像分别红外和可见光摄像机采集，需要将深度图像与 RGB 图像对齐，以便在同一坐标系下处理这两种类型的图像数据。<br>
下面就是创建Model类的实例：<br>
.self.model = Model() <br>
Model类是VAIC的核心，是深度测量和物体识别的神经网络模型，也是VAIC运用AI技术的重要部分。这里需要特殊提醒的是：<br>
class Model的定义在model.py文件中，overunder.py程序开始就通过from model import Model语句导入了其在model.py定义。<br>
同样我们首先查看model实例的__init__(self)函数。<br>

In [ ]:
def __init__(self):
        # Initialize the TensorRT engine and execution context.

        # Define file paths for ONNX and engine files
        current_folder_path = os.path.dirname(os.path.abspath(__file__))
        onnx_file_path = os.path.join(current_folder_path, "VEXOverUnder.onnx")  
        # If you change the onnx file to your own model, adjust the file name here
        engine_file_path = os.path.join(current_folder_path, "VEXOverUnder.trt")  
        # This should match the .onnx file name

        # Get the TensorRT engine
        self.engine = Model.get_engine(onnx_file_path, engine_file_path)

        # Create an execution context
        self.context = self.engine.create_execution_context()

        # Allocate buffers for input and output
        self.inputs, self.outputs, self.bindings, self.stream = 
                                                common.allocate_buffers(self.engine)

根据前面的介绍，为使用NVIDIA TensorRT进行高速推理,需要对神经网络进行优化和格式变换，将.onnx文件格式转化为.trt格式，前三行程序是检查onnx和trt文件是否存在，<br>
这里的.onnx文件就是VEX AI利用pytorch进行微调的YOLOv3神经网络，专用于VAIC_23_24赛季三色球物体和距离的识别。<br>
self.engine = Model.get_engine(onnx_file_path, engine_file_path)是格式转换函数。函数如下：<br>

In [ ]:
       def build_engine():
            print("Building engine file from onnx, this could take a while")
            # Builds and returns a TensorRT engine from an ONNX file.
            with trt.Builder(TRT_LOGGER) as builder, \
                    builder.create_network(common.EXPLICIT_BATCH) as network, \
                    builder.create_builder_config() as config, \
                    trt.OnnxParser(network, TRT_LOGGER) as parser, \
                    trt.Runtime(TRT_LOGGER) as runtime:

                config.max_workspace_size = 1 << 28  # Set maximum workspace size to 256MiB
                builder.max_batch_size = 1

                # Check if ONNX file exists
                if not os.path.exists(onnx_file_path):
                    print("ONNX file {} not found.".format(onnx_file_path))
                    exit(0)

                # Load and parse the ONNX file
                with open(onnx_file_path, "rb") as model:
                    if not parser.parse(model.read()):
                        print("ERROR: Failed to parse the ONNX file.")
                        for error in range(parser.num_errors):
                            print(parser.get_error(error))
                        return None

                # Set input shape for the network
                network.get_input(0).shape = [1, 3, 320, 320]

                # Build and serialize the network, then create and return the engine
                plan = builder.build_serialized_network(network, config)
                engine = runtime.deserialize_cuda_engine(plan)
                with open(engine_file_path, "wb") as f:
                    f.write(plan)
                return engine

        # Check if a serialized engine file exists and load it if so, otherwise build a new 
        # one
        if os.path.exists(engine_file_path):
            with open(engine_file_path, "rb") as f, trt.Runtime(TRT_LOGGER) as runtime:
                return runtime.deserialize_cuda_engine(f.read())
        else:
            return build_engine()

按照python对齐规则，程序从底部开始执行，如果.trt文件存在，打开.trt文件，导入TensorRT的runtime功能，使用deserialize_cuda_engine从序列化文件中加载CUDA引擎。<br>
在 get_engine()还定义了自用的build_engine()函数。如程序段所示，当在目录中没有找到.trt文件，调用此函数将目录的.onnx文件转换为.trt格式。以下对变换函数进行简单说明：<br>
.with语句引用了TensorRT的一些功能函数，builder、network、config、parser和runtime，为下面的格式变换做准备。<br>
.config.max_workspace_size = 1 << 28 定义网络内存空间大小。<br>
.builder.max_batch_size = 1 批量处理输为1，在训练时为提高效率，max_batch_size=64或更大，推理时只对一帧图像进行识别。<br>
接着是检查.onnx是否存在、读入文件并检验是否正确。<br>
.network.get_input(0).shape = [1, 3, 320, 320],指定网络输入格式为[批量输入=1，color=3(RGB)，图像宽=320，图像高=320]。<br>
.plan = builder.build_serialized_network(network, config) 格式变换函数，使用TensorRT的API来定义和配置网络模型,应用一系列优化策略，如层融合、内存管理优化等，<br>
以提高网络的执行速度和降低延迟,将优化后的网络结构及其相关信息保存到一个序列化（通常是二进制）内存中。<br>
.engine = runtime.deserialize_cuda_engine(plan)的功能则是从序列化内存中恢复（或反序列化）CUDA引擎。CUDA引擎是TensorRT中用于执行深度学习推理的神经网络,引擎可以直接用于后续的推理操作。<br>
最后将变换后网络数据保存为.trt文件，以便以后使用直接读入。<br>


In [ ]:
self.context = self.engine.create_execution_context()

为推理engine创建一个新的执行上下文，即为神经网络配置输入和输出连接。<br>

In [ ]:
self.inputs, self.outputs, self.bindings, self.stream = common.allocate_buffers(self.engine)                                             

为神经网络输入和输出分配内存。各内存的结构和作用在推理部分再加以介绍，这里common.allocate_buffers()内存分配函数在common.py定义，通过model.py文件开始部分的 <br>
import common<br>
语句导入。<br>
至此，作为VAIC的AI核心内容的神经网络的导入已经解释完毕。<br>


###   2. &nbsp; &nbsp;深度摄像机的配置与连接

回到overunder.py文件中class MainApp的def __init__(self)函数，为突出神经网络的核心作用，首先解释了YOLOv3网络的建立，现在开始介绍其外部设备Camera的配置和连接。<br>
同样在建立Camera的实例时:<br>

In [ ]:
self.camera = Camera()
self.camera.start()

执行Class Camera()的初始化函数。
注意以下程序段中的rs是表示使用了pyrealsence2库，并在overunder.py开始部分由<br>
import pyrealsense2 as rs
语句导入。<br>

In [ ]:
    def __init__(self):
        self.pipeline = rs.pipeline()  # Initialize RealSense pipeline
        self.config = rs.config()
        # Enable depth stream at 640x480 in z16 encoding at 30fps
        self.config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
        # Enable color stream at 640x480 in bgr8 encoding at 30fps
        self.config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

如注释所示分别建立rs.pipeline()和rs.config()的实例。<br>
配置深度数据流为宽度=640pix，高度=480pix，数据格式=z16，帧数=30/秒。<br>
配置图像数据流为宽度=640pix，高度=480pix，数据格式=bgr8，帧数=30/秒。<br>

In [ ]:
self.camera.start()

In [ ]:
    def start(self):
       self.profile = self.pipeline.start(self.config)   # Start the pipeline
        # Obtain depth sensor and calculate depth scale
        depth_sensor = self.profile.get_device().first_depth_sensor()
        self.depth_scale = depth_sensor.get_depth_scale()

.self.profile = self.pipeline.start(self.config) 按照刚才定义的深度和图像配置config，设定摄像机硬件参数，按配置的参数打开视频流<br>
.depth_sensor = self.profile.get_device().first_depth_sensor()从当前配置的 RealSense 设备中获取一个深度传感器对象<br>
.self.depth_scale = depth_sensor.get_depth_scale()获取深度传感器的深度比例。这个比例是一个浮点数，它表示从深度图像中的像素值到实际物理距离的转换关系。<br>
self.depth_scale是创建self.processing = Processing(self.camera.depth_scale)的参数。<br>
至此，Camera的设置和启动完成，通过pipeline按深度数据流为宽度=640pix，高度=480pix，数据格式=z16，帧数=30/秒、<br>
图像数据流为宽度=640pix，高度=480pix，数据格式=bgr8，帧数=30/秒,向Jetson Nano传送数据。<br>


###   3. &nbsp; &nbsp;VAIC神经网络的物体识别过程<br>
介绍YOLOv3神经网络物体识别推理过程，要回到主程序，这是整个AI控制程序的入口。

In [ ]:
if __name__ == "__main__":
    app = MainApp()  # Create the main application
    app.run()  # Run the application

在MainApp()的初始化中，我们暂时忽略:<br>
.self.v5 = V5SerialComms()<br>
.self.v5Map = MapPosition()<br>
.self.v5Pos = V5GPS()<br>
.self.v5Web = V5WebData(self.v5Map, self.v5Pos)<br>
.self.stats = Statistics(0, 0, 0, 640, 480, 0, False)<br>
.self.rendering = Rendering(self.v5Web)<br>
app.run()是一循环函数，在 app.run() 的，我们暂时忽略:<br>
.self.v5.start()<br>
.self.v5Pos.start()<br>
.self.v5Web.start()<br>
的说明，并不是这些功能不重要，任何一个功能不能正常执行，系统将不能工作，只因为这些功能都是常规的、辅助的，与AI物体识别技术无直接关系。<br>
都将在后续加以详细说明。
进入app.run()的循环：

In [ ]:
try:
    while True:
        start_time = time.time()  # start time of the loop
        frames = self.camera.get_frames()
        depth_image, color_image, depth_map = self.processing.process_frames(frames)
        invoke_time = time.time()
        output, detections = self.processing.detect_objects(color_image)
        invoke_time = time.time() - invoke_time
        aiRecord = self.processing.compute_detections(self, detections, depth_image)
        self.set_v5(aiRecord)
        self.rendering.set_images(output, depth_map)
        self.rendering.set_detection_data(aiRecord)
        self.rendering.set_stats(self.stats, self.v5Pos, start_time, invoke_time, run_time)
        # self.rendering.display_output(output)
finally:
    self.camera.stop()

.start_time = time.time()保存每次循环的开始时间。<br>
.frames = self.camera.get_frames()从pipeline取出帧图像：<br>
  &nbsp; &nbsp;  def get_frames(self):<br>
    &nbsp; &nbsp; &nbsp; &nbsp;   return self.pipeline.wait_for_frames()<br> 
wait_for_frames()的意思是在等待图像帧刷新、两帧间隔之间取出图像和距离数据。<br>
.depth_image, color_image, depth_map = self.processing.process_frames(frames)函数如下所示：<br>
所谓帧对齐，是把红外测距的深度矩阵按摄像机彩色变换图像位置一一对齐，为场景图像的每一点给出一个深度值，场景图像在color_image,<br>
深度保存depth_image。利用depth_image，进行归一化，应用COLORMAP_JET颜色映射，生成一个彩色的深度图(depth_map)。<br>
函数的作用是从对齐的帧中提取深度图像和颜色图像，然后对深度图像进行归一化和颜色映射，并返回这三个图像。<br>

In [ ]:
    def process_frames(self, frames):
        # Align frames and extract color and depth images
        # Apply a color map to the depth image
        self.align_frames(frames)
        depth_image = np.asanyarray(self.depth_frame_aligned.get_data())
        color_image = np.asanyarray(self.color_frame_aligned.get_data())
        depthImage = cv2.normalize(depth_image, None, alpha=0.01, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
        depth_map = cv2.applyColorMap(depthImage, cv2.COLORMAP_JET)

        return depth_image, color_image, depth_map

In [ ]:
        invoke_time = time.time()
        output, detections = self.processing.detect_objects(color_image)
        invoke_time = time.time() - invoke_time

两次invoke_time之差是一次物体检测的时间。<br>
.output, detections = self.processing.detect_objects(color_image)是检测函数，输入的是彩色图像，输出的是检测结果。<br>

In [ ]:
    def detect_objects(self, color_image):
        # Perform object detection and return results using the Model class in model.py
        output, detections = self.model.inference(color_image) 
        return output, detections

而output, detections = self.model.inference(color_image)调用了下列函数。inference是训练完成的神经网络进行正向转播的过程，也就是神经网络的工作状态，需要比较详细的解释。<br>
这里列出了函数的全部语句，并逐条进行分析。<br>

In [ ]:
def inference(self, inputImage):
    # Perform inference on the given image and return the bounding boxes, scores, and 
    # classes of detected objects.

    # Process the input image
    input_image = Model.image_processing(inputImage)

    # Define input resolution and create preprocessor
    input_resolution_yolov3_HW = (320, 320)
    preprocessor = PreprocessYOLO(input_resolution_yolov3_HW)

    # Process the image and get original shape
    image_raw, image = preprocessor.process(input_image)
    shape_orig_WH = image_raw.size

    # Define output shapes for post-processing
    output_shapes = [(1, 24, 10, 10), (1, 24, 20, 20)]

    # Set the input and perform inference
    self.inputs[0].host = image
    trt_outputs = common.do_inference_v2(self.context, bindings=self.bindings, 
                inputs=self.inputs,outputs=self.outputs,stream=self.stream)
                                            
                                         

        # Reshape the outputs for post-processing
        trt_outputs = [output.reshape(shape) for output, shape in zip(trt_outputs, 
                                                                      output_shapes)]

        #Define arguments for post-processing
        postprocessor_args = {
            "yolo_masks": [(3, 4, 5), (0, 1, 2)],
            "yolo_anchors": [(10, 14), (23, 27), (37, 58), (81, 82), (135, 169), (344, 319)],
            "obj_threshold": [0.4, 0.90, 0.90],  # Different thresholds for each class
            #label (Green, Red, Blue)
            "nms_threshold": 0.5,
            "yolo_input_resolution": input_resolution_yolov3_HW,
        }#

        # Perform post-processing
        postprocessor = PostprocessYOLO(**postprocessor_args)
        boxes, classes, scores = postprocessor.process(trt_outputs, (shape_orig_WH))

        Detections = []

        # Handle case with no detections
        if boxes is None or classes is None or scores is None:
            print("No objects were detected.")
            return input_image, Detections

        # Draw bounding boxes and return detected objects
        obj_detected_img = Model.draw_bboxes(image_raw, boxes, scores, classes, 
                                             ALL_CATEGORIES, Detections)
        obj_detected_img = obj_detected_img.convert("RGB")
        return np.array(obj_detected_img), Detections


1..input_image = Model.image_processing(inputImage)，它的目的是通过调整图像的色调、饱和度和亮度来增强图像。<br>
.hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)转换图像到HSV颜色空间，使用OpenCV的cvtColor函数将输入的BGR图像（OpenCV默认读取的图像格式）转换为HSV颜色空间。<br>
HSV颜色空间由色调（Hue）、饱和度（Saturation）和亮度（Value）三个分量组成，更适用于处理颜色信息。<br>
.hsv[..., 0] = hsv[..., 0] + 12修改色调、饱和度和亮度通道,将HSV图像中的色调通道的每个像素值增加12。这通常会改变图像的整体颜色，使其向某种特定的色调偏移。<br>
.hsv[:, :, 1] = np.clip(hsv[:, :, 1] * 1.2, 0, 255)首先乘以1.2来增加饱和度通道的值，然后使用np.clip函数确保这些值在0到255的范围内。增加饱和度通常会使图像的颜色更鲜艳。<br>
.hsv[:, :, 2] = np.clip(hsv[:, :, 2] * 1.1, 0, 255)类似地调整亮度通道。乘以1.1会提高亮度，但同样，我们使用np.clip确保这些值不会超出有效范围。<br>
.return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)将图像转换回BGR颜色空间.<br>
经过上述修改后，hsv变量现在包含修改后的HSV图像。为了能在常规的显示器或应用程序中显示，我们需要将其转换回BGR格式。使用cvtColor函数将HSV图像转换回BGR格式，并返回这个处理后的图像。<br>
2.
.input_resolution_yolov3_HW = (320, 320)<br>
.preprocessor = PreprocessYOLO(input_resolution_yolov3_HW)<br>
根据YOLOv3的输入图像分辨率的标准，创建320X320分辨率的Class PreprocessYOLO的实例，详细内容请参考data_processing.py。<br>
3.
.image_raw, image = preprocessor.process(input_image)<br>
.shape_orig_WH = image_raw.size<br>
preprocessor.process()调用了_load_and_resize(self, input_image)和_shuffle_and_normalize(self, image)函数，将输入图像变换为320X320<br>
PIL格式用于推理的image。image_raw变换为PIL格式，保留原分辨率用于动态显示,细节请参考data_processing.py。<br>
4.
.output_shapes = [(1, 24, 10, 10), (1, 24, 20, 20)]定义YOLOv3的输出格式。包含了YOLOv3模型输出的形状。YOLOv3模型通常会有多个输出层，每个输出层对应不同尺度的特征图。<br>
在这里有两个输出层，它们的形状分别是(1, 24, 10, 10)和(1, 24, 20, 20)。这些形状的含义通常是：<br>
第一个维度（1）是预测批量大小（batch size），表示一次处理多少张图像（在这里是1张）。<br>
第二个维度（24）是每层的锚点（anchor）的数量乘以预测物体参数数量（物体中心坐标系x,y、预测边框w,h、置信度confidence和三色球的分类概率class probabilities，3X(5+3)=24）。关于锚点（anchor）下文将进行定义和说明。<br>
剩下的两个维度（10x10和20x20）是特征图的尺寸，表示在不同尺度上的网格数量。<br>

In [ ]:
input_resolution_yolov3_HW = (320, 320)
preprocessor = PreprocessYOLO(input_resolution_yolov3_HW)

In [ ]:
 # Set the input and perform inference
        self.inputs[0].host = image
        trt_outputs = common.do_inference_v2(self.context, bindings=self.bindings,
                     inputs=self.inputs,outputs=self.outputs, stream=self.stream)
                                             
        # Reshape the outputs for post-processing
        trt_outputs = [output.reshape(shape) for output, shape in zip(trt_outputs, 
                                                        output_shapes)]

用视觉神经网络YOLOv3对实时获取的场景图像进行物体识别。<br>                                         
1.<br> 
.self.inputs[0].host = image 图像送达为推理分配的CPU端的内存空间，在CUDA平台编程里，用host表示CPU，driver表示GPU。<br>
2.<br> 
.trt_outputs = common.do_inference_v2(self.context, bindings=self.bindings, inputs=self.inputs,outputs=self.outputs, stream=self.stream)<br> 
调用了由NVIDIA提供的common.do_inference_v2()函数，trt_outputs 是从YOLOv3模型获得的原始输出，通常是一个包含多个列表或元组，包含了模型对输入图像的检测结果。<br>
self.context是上面介绍过的YOLOv3网络、bindings是主机内存CPU和设备内存GPU之间的数据绑定起来的数组，确保了在执行推理时，输入数据正确地被传输到设备，并且输出数据被正确地传回主机。<br>

参看common.py文件里do_inference_v2()函数，会看到涉及CUDA平台实现TensorRT的一系列计算操作，这里只进行计算的简略说明。<br> 
.[cuda.memcpy_htod_async(inp.device, inp.host, stream) for inp in inputs],在CUDA平台编程里，将检测图像从CPU端复制到执行YOLOv3网络TensorRT正向传播的GPU端。<br> 
.context.execute_async(batch_size=batch_size, bindings=bindings, stream_handle=stream.handle),执行推理计算。<br>
.[cuda.memcpy_dtoh_async(out.host, out.device, stream) for out in outputs],识别结果从GPU的 out.device复制到CPU的out.host。<br>
.stream.synchronize(),数据同步化，等待并检查所有在GPU上的操作完成。<br>
这里需要说明的是，前面执行的函数都带有_async，为异步执行函数。异步编程（Asynchronous Programming）是一种编程技术，它允许程序在等待某个操作（如I/O操作）完成时继续执行其他任务，<br>
而不是阻塞并等待该操作完成。这种技术可以显著提高程序的响应性和吞吐量，尤其是在处理CPU和GPU联动、网络请求、文件读写等需要等待的操作时。<br>
调用 stream.synchronize()，等待所有在GPU排队的操作完成。这是必要的，因为虽然前面的内存复制和推断操作是异步的，需要在推理结束之前确保它们已经正确完成。<br>
.trt_outputs = [output.reshape(shape) for output, shape in zip(trt_outputs, output_shapes)]
将trt_outputs列表中的每一个元素（假设是NumPy数组或类似的可迭代对象）根据对应的output_shapes列表中的形状（shape）进行重塑。<br>
重塑后的数据被收集到一个新的列表中，赋值给trt_outputs。注意，这里实际上是按output_shapes的形状原地修改了trt_outputs列表。<br>

首先要对后处理参数postprocessor_args进行解释,如下所示：<br>
.yolo_masks (yolo掩码):YOLOv3模型使用了不同尺度的特征图来检测不同大小的对象。每个特征图会负责检测特定大小的对象。<br>
yolo_masks定义了每个特征图对应的锚点（anchor）的索引。在这个例子中，有两个特征图，第一个特征图使用锚点索引(3, 4, 5)，而第二个特征图使用锚点索引(0, 1, 2)。<br>
.yolo_anchors(yolo锚点):锚点是预先定义好的一组宽高比，用来作为预测物体边界框的初始形状。YOLOv3使用这些锚点来预测实际边界框相对于这些初始形状的偏移量。这里定义了6个锚点，每个锚点是一个宽度和高度的二元组。<br>
通俗地解释：所谓锚点就是给出一个接近被识别物体形状的矩形，以区别不同长宽比例的其他物体。对于要识别的三色球，其宽高比为1:1，摄像机的图像是640X480，<br>
由于YOLOv3的输入图像为320X320，宽度被压缩为1/2，高度压缩为320/480，识别物体的边框变成长方形。<br>
<br>
所谓锚点是指要识别物体大致宽高比，所谓掩码是按大小分层锚点的索引。这里掩码分两层,大尺度的特征图(3,4,5)对应锚点(81, 82), (135, 169), (344, 319);小尺度的特征图(0, 1, 2)<br>
对应锚点(10, 14), (23, 27), (37, 58)。<br>
.obj_threshold: [0.4, 0.90, 0.90]物体颜色的判断阈值；绿=0.4，红=0.9，蓝=0.9。<br>
.nms_threshold:非极大值抑制（NMS）阈值;NMS是为了减少重叠边界框数量的技术。当两个边界框的重叠区域（IOU，即交并比）大于NMS阈值时，会保留置信度较高的那个边界框，抑制（即删除）信度较低；当两个边界框很接近时，被认为就是一个物体，可以避免一个物体被多次识别。<br>
.yolo_input_resolution: input_resolution_yolov3_HW,320X320,YOLOv3输入的图像尺寸。<br>
.postprocessor = PostprocessYOLO(**postprocessor_args) 将以上数组作为参数创建Class PostprocessYOLO 实例。<br>
.boxes, classes, scores = postprocessor.process(trt_outputs, (shape_orig_WH)) 从TensorRT前向传递生成的YOLOv3输出中，对这些输出进行后处理，并返回检测到的对象的边界框列表以及它们各自的类别和置信度，并分别放在不同的列表中。函数的boxes, classes, scores是从postprocessor.process方法的返回值中解包的三个变量。<br>
即目标检测模型的输出中提取出的边界框（bounding boxes）、类别（classes）和置信度分数（scores）。shape_orig_WH包含了原始输入图像的宽度和高度（以像素为单位），这个信息在将模型输出中的边界框坐标从模型内部的归一化，坐标或相对坐标转换回原始图像的坐标时非常重要。<br>

In [ ]:
postprocessor_args = {
    "yolo_masks": [(3, 4, 5), (0, 1, 2)],
    "yolo_anchors": [(10, 14), (23, 27), (37, 58), (81, 82), (135, 169), (344, 319)],
    "obj_threshold": [0.4, 0.90, 0.90],  # Different thresholds for each class label 
    #(BlueGreen, Red, )
    "nms_threshold": 0.5,
    "yolo_input_resolution": input_resolution_yolov3_HW,
}
postprocessor = PostprocessYOLO(**postprocessor_args)
boxes, classes, scores = postprocessor.process(trt_outputs, (shape_orig_WH))

Detections = []

# Handle case with no detections
if boxes is None or classes is None or scores is None:
    print("No objects were detected.")
    return input_image, Detections

# Draw bounding boxes and return detected objects
obj_detected_img = Model.draw_bboxes(image_raw, boxes, scores, classes, ALL_CATEGORIES, 
                                     Detections)
obj_detected_img = obj_detected_img.convert("RGB")
return np.array(obj_detected_img), Detections

<img src="sigmoid-1.png" width="400" height="300" >


    图-1 sigmoid函数



参看data_processing.py中class PostprocessYOLO的 def process(self, outputs, resolution_raw)函数,依次调用了_reshape_output、_process_feats、_process_feats等函数。无非是对输出的识别张量进行包括转置和形变的变换，按识别框、类别和置信度各自排列输出识别物体的信息。这里就不做详细介绍，对python张量计算有兴趣的读者可自行参考。<br>
需要特殊说明的是sigmoid激活函数的计算，sigmoid 函数是一个在机器学习和深度学习中常用的激活函数，$$\\sigmoid(x) = 1/{(1 + e^{-x})}$$ 
特别是在逻辑回归和神经网络中。它可以将任何实数映射到 0 到 1 的范围内，因此经常被用作输出层的激活函数，以将输出解释为概率，前面介绍过每一个三色球有三个分类，经过sigmoid函数处理，成为各个分类的判断概率。函数图像如图-1所示。<br>
最后，调用Model自身的draw_bboxes函数进行识别物体的最后处理。<br>
.obj_detected_img = Model.draw_bboxes(image_raw, boxes, scores, classes, ALL_CATEGORIES, Detections)<br>
函数定义如下，函数里使用了python的PIL（Python Imaging Library）图像处理函数库，在model.py的文件开始出，由语句from PIL import ImageDraw导入。<br>
取出每一个识别物体的中心和识别框的对角线坐标，把每一个物体的识别数据整理归纳到Detections数组里。max和min函数控制当图像上只有部分物体时识别框超出图像范围，取图像边缘。
draw_bboxes完成后，调用obj_detected_img = obj_detected_img.convert("RGB")将检测图像转换为RGB格式，以便后续程序使用。这样识别过程完成。<br>

In [ ]:
def draw_bboxes(image_raw, bboxes, confidences, categories, all_categories, Detections, 
                bbox_color="white"):
        # Draw bounding boxes on the original image and return it.

        # Create drawing context
        draw = ImageDraw.Draw(image_raw)

        # Draw each bounding box
        for box, score, category in zip(bboxes, confidences, categories):
            x_coord, y_coord, width, height = box
            left = max(0, np.floor(x_coord + 0.5).astype(int))
            top = max(0, np.floor(y_coord + 0.5).astype(int))
            right = min(image_raw.width, np.floor(x_coord + width + 0.5).astype(int))
            bottom = min(image_raw.height, np.floor(y_coord + height + 0.5).astype(int))

            # Draw the rectangle and text
            # draw.rectangle(((left, top), (right, bottom)), outline=bbox_color)
            # draw.text((left, top - 12), "{0} {1:.2f}".format(all_categories[category], score), 
            # fill=bbox_color)

            # Create and store the raw detection object
            raw_detection = rawDetection(int(left), int(top), [x_coord, y_coord], int(width), 
                                         int(height), score,category)
                                         
            Detections.append(raw_detection)

        return image_raw

至此，完成了output, detections = self.processing.detect_objects(color_image)程序的整个执行过程，这也是VAIC_23_24引进AI技术的主要内容。<br>
需要提示的是每一个识别物体按：<识别框的左边位置，识别框的上边位置，[中心x坐标，中心y坐标]，识别框宽度，识别框高度，识别置信度，识别类别>数据格式组合在Detections数组中，<br>
下一节将介绍如何用AI识别物体的数据结合Vex Brain送来的VEX GPS实时场地定位，换算为世界坐标系坐标反馈给Brian，对Vex 5机器人进行控制。<br>

###   4.&nbsp; &nbsp;物体识别数据的处理<br>

回到overrunder.py主程序，在class MainApp的初始化中先对前面暂时搁置的其他功能进行补充说明：<br>
    .self.v5 = V5SerialComms(),建立Jetson对Brian识别物体数据传输以及Brian对Jetson的VexGPS场地定位位置数据的USB通道。<br>
    .self.v5Map = MapPosition(),提供识别数据从摄像机坐标到场地物理坐标变换的Tait-Bryan计算矩阵、摄像机安装位置的计算参数和计算方法。<br>
    .self.v5Pos = V5GPS()，在Jetson端创建V5GPS实例、创建接收从Brian传送GPS数据的线程，用于运行GPS数据读取和处理包括建立连接、数据读取、坐标转换以及状态处理，<br>
     设置GPS偏移量的本地变量。<br> 
    .self.v5Web = V5WebData(self.v5Map, self.v5Pos)，建立WEB服务器，实时传送更新数据，为远程终端实时显示识别物体的场地视场下的坐标和类别和赛场实况。<br>
    .self.stats = Statistics(0, 0, 0, 640, 480, 0, False),初始化统计属性，如帧率（FPS）、CPU温度、视频尺寸、运行时间以及GPS连接状态。<br>
    .self.rendering = Rendering(self.v5Web),通过WEB链接，发送Jetson端场地的实时图像和物体识别数据，同步显示在远程终端上。如图2所示。<br>
       在V5Comm.py文件中定义了与以上功能的相关数据结构和方法，以便在实现功能时引用。对以下功能进行说明时，如有必要再行说明。<br>
       有必要简单解释接收Brian发送到Jetson的GPS数据、Jetson发送到Brian的aiRecord位置数据的传输机理，两者均通过线程进行。<br>
       所谓线程(thread)是一种实时运行的后台程序,一经启动，就能自动运行。<br>
       除共用电源并安装同一机器人小车上，GPS数据发送与接收，AI识别物体位置数据的发送与接收是Brian和Jetson两个系统的唯一联系。<br>
       GPS数据的发送端是Brian，接收端是Jetson，而aiRecord识别物体位置数据的发送端是Jetson，接收端是Brian。在V5Comm.py和V5Position.py文件都调用了<br>
       self.__thread = threading.Thread(target=self.__run, args=())，这是调用线程目标函数__run()创建各自的线程,而各自的__run()具有不同功能。<br>
       忽略线程中的其他辅助功能解释， V5GPS的__run()函数里是data = self.__ser.read_until(b'\xCC\x33')，表示接收数据直到出现结尾符b'\xCC\x33'；<br>
       简单解释V5SerialComms的__run()函数数据传输过程，实现数据发送的程序段如下：<br>
         首先线程读取Brian端是否有数据传输要求。<br>
        .data = self.__ser.readline().decode("utf-8").rstrip()，从端口读出Brian端送来的命令，命令 "AA55CC3301"出现，就是Brian端要求传送数据。<br>
        忽略线程时间片请求，释放互锁等常规操作，<br>
      .self.__ser.write(data) 表示将data数据全部送往Brian。<br>
       Brian端何时需要、如何发出数据请求命令等技术细节将在下面brian端Cpp程序讲解时再行介绍。

In [ ]:
    while self.__started:  # Continue reading while thread is started
        # Read data from the serial port
        data = self.__ser.readline().decode("utf-8").rstrip()
        if(data == "AA55CC3301"):
            self.__detectionLock.acquire()
            myPacket = V5SerialPacket(self.__MAP_PACKET_TYPE, self.__detections)
            self.__detectionLock.release()
            data = myPacket.to_Serial()
            self.__ser.write(data)  # Write serialized data to the serial port

   GPS数据的发送由Brian上运行的CPP程序完成，Jetson端的线程接收。同样忽略线程中的其他辅助功能解释，接收程序段如下：<br>
   .data = self.__ser.read_until(b'\xCC\x33'),线程程序读端口直到出现结束符'\xCC\x33'。<br>
   .x, y, z, az, el, rot = struct.unpack(b'hhhhhh', data[2:14]),从接收的数据包解出位置和角度数据。<br>

In [ ]:
    data = self.__ser.read_until(b'\xCC\x33')
    if(len(data) == 16):
        self.__frameCount = self.__frameCount + 1
        status = data[1]
        x, y, z, az, el, rot = struct.unpack('<hhhhhh', data[2:14])

现在讨论对识别物体数据的后续处理，主循环程序app.run() 在完成物体识别processing.detect_objects(color_image)后执行<br>

In [ ]:
aiRecord = self.processing.compute_detections(self, detections, depth_image)

如overunder.py的def compute_detections(self, v5, detections, depth_image)函数注释所述，创建一个AIRecord，并使用深度和图像数据计算检测结果。<br>
aiRecord是Jetson发送给Brian的控制信息的数据结构，aiRecord.detections包含了摄像机视场内每个检测到的目标的ClassID（类别ID）、Probability（概率）和深度信息，<br>
除此之外，还包含检测相机图像和目标在场地的位置信息。程序的列表如下：<br>

In [ ]:
def compute_detections(self, v5, detections, depth_image):
        # Create AIRecord and compute detections with depth and image data.
        # Each AIRecord contains the ClassID, Probablity, and depth information for each detection
        # In addition to the detection's camera image and map position information.
        aiRecord = V5Comm.AIRecord(v5.get_v5Pos(), [])
        for detection in detections:
            depth = self.get_depth(detection, depth_image)
            imageDet = V5Comm.ImageDetection(
                int(detection.x),
                int(detection.y),
                int(detection.Width),
                int(detection.Height),
            )
            mapPos = v5.v5Map.computeMapLocation(detection, depth, aiRecord.position)
            mapDet = V5Comm.MapDetection(mapPos[0], mapPos[1], mapPos[2])
            detect = V5Comm.Detection(
                int(detection.ClassID),
                float(detection.Prob),
                float(depth),
                imageDet,
                mapDet,
            )
            aiRecord.detections.append(detect)
        return aiRecord

aiRecord = V5Comm.AIRecord(v5.get_v5Pos(), []),参看V5Comm.py的AIRecord定义和初始化：<br>

    

In [ ]:
def __init__(self, position: Position, detections: "list[Detection]")

第一参数由v5.get_v5Pos()取得的Position，第二参数为"list[Detection]"，在创建aiRecord实例时就引进了两个变量，VexGPS传来的场地定位数据position和YOLOv3推理得到的物体识别数据detections。<br>
上节已经解释在数据detections数组包含了摄像机视场识别的多个物体(三色球)，下述循环计算每一识别物体的距离。<br>
从VEXGPS取得的数据position包括:
1. frameCount 数据帧计数。<br>
2. status 通讯状态，状态正常表示数据的可靠。<br>
3. x  机器人旋转中心在场地上的X坐标。<br>
4. y  机器人旋转中心在场地上的Y坐标。<br>
5. z  机器人旋转中心在场地上的Z坐标。<br>
6. azimuth   方位角，方位角是从某个参考方向（通常是北方或正南方）开始，沿地平面顺时针测量到某一点的角度。在导航中，方位角用来确定一个点相对于观察者的方向。<br>
7. elevation 高度角，高度角是从地平线向上到目标点的角度，与方位角结合使用可以精确地定位天体或其他对象。在航空和卫星通信中，高度角用于描述飞行器或卫星相对于地平线的倾斜程度。<br>
8. rotation  扭转角，扭转角是关于观察轴的旋转，表示从地平面指向天顶的轴线绕自身的旋转。在摄影测量、机器视觉和机器人技术中，用于校正调整图像和传感器的方x向。<br>
这些数据使用，在坐标变换计算时在详细介绍。<br>

        

In [ ]:
self.frameCount = frameCount
self.status = status
self.x = x
self.y = y
self.z = z
self.azimuth = azimuth
self.elevation = elevation
self.rotation = rotation

在帧对齐函数中，相对于彩色场景图像每一点的深度(距摄像头成像屏的距离)已经保存在depth_image中。<br>
get_depth(detection, depth_image) 函数截取识别物体识别框中心位置百分之十区域的depth_image深度数据进行算术平均，作为此物体的距离数据，
(参看overunder.py的def get_depth(self, detection, depth_img)函数)。<br>
imageDet是被识别物体的图像范围，中心坐标(x,y)和识别框(Width,Height),作为传递给Brian的数据的一部分。<br> 

In [ ]:
mapPos = v5.v5Map.computeMapLocation(detection, depth, aiRecord.position)

计算传输给Brian的机器人各个识别物体在场地世界坐标系的数据。请参照V5MapPosition.py的def computeMapLocation(self, detection, depth, position)函数。 <br>
通过将屏幕中识别物体的相对位置与Brian GPS场地位置角度变换矩阵相乘，矩阵乘法的结果给出了识别物体相对于机器人在场地3D空间中的物理位置。<br>
下面对此函数进行简单的解释：
computeMapLocation()程序开始是一些常数的设定，摄像机和GPS在Vex机器人上的长度量纲及位置数据从camera_offsets.json和gps_offsets.json读出，请参照V5Web.py的class V5WebData的__init__(self, v5Map, v5Pos, port = None)函数定义及主程序self.v5Web = V5WebData(self.v5Map, self.v5Pos)。
这些数据由摄像机和GPS设备在机器人小车上安装的位置确定，并由设计者预先测量并记入上述文件中。<br>
接下来是创建旋转变换矩阵，由方位角（Azimuth）、高度角（Elevation）、扭转角（Twist）组成的旋转计算方式称为Tait-Bryan旋转复合矩阵。<br> 

Tait-Bryan旋转复合矩阵（也被称为欧拉旋转矩阵）在多个领域都有应用，特别是在描述三维空间中物体或坐标系的旋转时。以下是几个具体的应用场景：<br>
    航空和航天工程：在航空和航天领域，飞机的姿态通常使用Tait-Bryan角（偏航角、俯仰角和滚转角）来描述。这三个角度分别对应于飞机绕其垂直轴、横轴和纵轴的旋转。通过Tait-Bryan旋转复合矩阵，可以方便地计算出飞机在不同姿态下的方向，以及进行姿态控制和导航。<br>
    机器人学：在机器人学中，机器人的末端执行器（如手臂、夹爪等）需要在三维空间中进行精确的定位和旋转。通过Tait-Bryan旋转复合矩阵，可以计算出末端执行器相对于机器人基座的姿态，并实现精确的位姿控制。<br>
    计算机图形学：在计算机图形学中，物体在三维场景中的旋转和变换是一个重要的主题。Tait-Bryan旋转复合矩阵可以用于计算物体在三维空间中的旋转矩阵，进而实现物体的旋转和变换。这在3D游戏、虚拟现实和动画等领域中都有广泛的应用。<br>
    物理模拟：在物理模拟中，如分子动力学模拟、刚体动力学模拟等，需要模拟物体在三维空间中的运动和旋转。Tait-Bryan旋转复合矩阵可以用于计算物体在不同时刻的姿态，进而模拟物体的运动和旋转过程。<br>
    总之，Tait-Bryan旋转复合矩阵在描述三维空间中物体或坐标系的旋转时具有广泛的应用。它提供了一种方便、高效的方式来计算旋转矩阵，并实现了物体在三维空间中的精确定位和旋转控制。<br>

Tait-Bryan旋转矩阵：<br><br><br>
$
R_{az,el,tw}= 
$
<br><br>
$
\begin{bmatrix}
\cos(az)\cos(tw) + \sin(az)\sin(el)\sin(tw) & \sin(az)\cos(el) & \cos(az)\sin(tw) - \sin(az)\sin(el)\cos(tw) \\
-\sin(az)\cos(tw) + \cos(az)\sin(el)\sin(tw) &
\cos(az)\cos(el) &
 -\sin(az)\sin(tw) - \cos(az)\sin(el)\cos(tw) \\
-\cos(el)\sin(tw) &
\sin(el) &
\cos(el)\cos(tw)
\end{bmatrix} 
$

在摄像机坐标系计算目标物的位置坐标向量。

In [ ]:
vector = np.zeros(shape=(3, 1))
vector[0] = depth * (detection.Center[0] - MAXSCREENX) / REALDIST
vector[1] = depth
vector[2] = depth * (MAXSCREENY - detection.Center[1]) / REALDIST

距离向量与变换矩阵的乘积，将向量变换到世界空间。通过将屏幕上物体的相对位置与机器人的视角信息相乘，矩阵乘法的结果给出了识别物体相对于机器人三维空间中的物理位置。<br>

In [ ]:
mapLocation = np.matmul(rot, vector)

叠加场地上当前机器人位置坐标，转换为世界坐标系。这时位置坐标是相对于摄像机屏中点。<br>

In [ ]:
mapLocation[0] += position.x
mapLocation[1] += position.y
mapLocation[2] += position.z

用摄像机安装位置对机器人回转中心的x、y、z距离([CAMERAOFFSETX], [CAMERAOFFSETY], [CAMERAOFFSETZ])再作一次变换，将位置坐标调整到机器人回转中心。<br>

In [ ]:
 cameraOffset = np.array([[CAMERAOFFSETX], [CAMERAOFFSETY], [CAMERAOFFSETZ]])
rotatedCameraOffset = np.matmul(rot, cameraOffset)

        # Add the adjusted camera offset
mapLocation[0] += rotatedCameraOffset[0]
mapLocation[1] += rotatedCameraOffset[1]
mapLocation[2] -= rotatedCameraOffset[2]  # Subtract Z offse

计算完成，返回机器人小车回转中心在赛场坐标系的位置最后返回mapPos=(mapLocation[0],mapLocation[1],mapLocation[2])。<br>

In [ ]:
 return mapLocation

回到主程序的compute_detections循环中:<br>
将当前识别物体的分类、置信度、距离深度、识别物体图像中心坐标和识别框宽高(摄像机视场坐标)、识别物体世界(竞赛场地)坐标组合到一个detect。<br>

In [ ]:
mapDet = V5Comm.MapDetection(mapPos[0], mapPos[1], mapPos[2])
detect = V5Comm.Detection(
    int(detection.ClassID),
    float(detection.Prob),
    float(depth),
    imageDet,
    mapDet,
                           )
    aiRecord.detections.append(detect)
    return aiRecord



回到主程序的compute_detections循环中:<br>
.mapDet = V5Comm.MapDetection(mapPos[0], mapPos[1], mapPos[2])，数组mapDet是识别物体场地位置坐标(X,Y,Z)。 <br> 
.detect = V5Comm.Detection(<br>
             int(detection.ClassID),<br>
             float(detection.Prob),<br>
             float(depth),<br>
             imageDet,<br>
             mapDet,<br>
           )<br>
将当前识别物体的分类、置信度、距离深度、识别物体图像中心坐标和识别框宽高(摄像机视场坐标)、识别物体世界(竞赛场地)坐标组合到一个detect。<br>
.aiRecord.detections.append(detect),追加到aiRecord.detections中。<br>
上述循环程序完成后，当前摄像机图像视场中识别物体的分类识别、位置坐标都存入aiRecord.detections中，准备通过USB连接发送至Brian。<br>

主程序的app.run()的self.set_v5(aiRecord)是发送程序，由于与Brian端CPP的接收程序有关，需要预先进行一些说明。<br>

In [ ]:
   def set_v5(self, aiRecord):
        # Set detection data to the Brain if it is connected but does not set any data if None
        if self.v5 is not None:
            self.v5.setDetectionData(aiRecord)

set_v5(self, aiRecord)调用了V5Comm.py的setDetectionData函数。如前所述通过V5SerialComms实例建立了向Brian发送数据的线程。<br>
self.v5.setDetectionData(self, data: AIRecord)是V5SerialComms中的方法(函数)，可以对自己的线程直接进行操作，参数data为AIRecord。<br>
线程调用 acquire 方法是试图获取锁。如果锁当前没有被其他线程持有（即锁是空闲的），该线程获得了锁，安全地发送数据。<br>
如果锁已经被其他线程持有，则 acquire 方法会在此循环调用它的线程，直到锁被释放（release），再发送数据。数据发送完成，即时释放。<br>

In [ ]:
    def setDetectionData(self, data: AIRecord):
        # Aquire lock and set detection data
        self.__detectionLock.acquire()
        self.__detections = data
        self.__detectionLock.release()

###    5.&nbsp; &nbsp;Jetson远程显示及V5Web.py程序
为了参赛者在比赛中能动态监视赛场情况，VAIC_23_24设置了实时远程显示模块，overrunder.pyclass Rendering在主程序竞赛循环中不断提供远程显示的实时数据，网络传输功能定义在V5Web.py文件。VAIC_23_24的远程通信使用了Websocket方式(参照文件开始的from websocket_server import WebsocketServer)。<br>
Websocket通信是计算机和互联网通用标准协议，程序也是按照Websocket标准编程，网络传输实现的基本方法做一些简单介绍。<br>
.self.__server.run_forever(True) 启动 Websocket 服务器的事件循环，让它开始监听连接和处理传入的消息。<br>
在Websocket运行机制中，常见的模式是服务器设置回调函数来处理从客户端接收到的消息，回调函数会根据客户端发送的命令或消息内容来处理相应的逻辑，并可能向客户端发送请求的数据作为响应。<br>
__new_client: 当有新客户端连接时调用。<br>
__client_left: 当客户端断开连接时调用。<br>
__message_received: 当服务器接收到来自客户端的消息时调用。<br>
在主程序class MainApp创建的初始化时，创建V5WebData实例引进参数了v5Map,v5Pos是为了设定v5Map的CameraOffset和v5Pos的GPSOffset。<br>

In [ ]:
self.v5Web = V5WebData(self.v5Map, self.v5Pos)

然后创建WebsocketServer服务器，定义了三个回调函数：

In [ ]:
        self.__server = WebsocketServer(host = '0.0.0.0', port = self.__serverPort, loglevel = logging.INFO) 
        self.__server.set_fn_new_client(self.__new_client)
        self.__server.set_fn_client_left(self.__client_left)
        self.__server.set_fn_message_received(self.__message_received)

.self.__server.set_fn_new_client(self.__new_client)，创建远程显示客户端，向服务器端发出数据请求，接收服务器端数据，显示赛场实时动态画面。<br>
.self.__server.set_fn_client_left(self.__client_left),监视传输通道状态，断开时给出报警。<br>
.__message_received(self.__message_received),接收到客户端发出的命令后，按照命令调用相应函数向远程服务器端发出显示内容。<br>
下面截取message_received部分程序段进行简单解释：<br>
g_pos、g_detect、g_stats、g_depth、g_color、get_camera_offset、get_gps_offset等是客户端向服务器端发送的数据请求命令(VAIC_23_24程序对此无任何说明)，如下所示，调用相应函数返回相应类型数据。

In [ ]:
    
def __message_received(self, client, server, message):
    .......
    cmds = message.split(",")
    for cmd in cmds:
        if(cmd == "g_pos"):
            outData['Position'] = self.__getPositionElement()
        elif(cmd == "g_detect"):
            outData['Detections'] = self.__getDetectionElement()
        elif(cmd == "g_stats"):
            outData['Stats'] = self.__getStatsElement()
        elif(cmd == "g_depth"):
            outData['Depth'] = self.__getDepthElement()
        elif(cmd == "g_color"):
            outData['Color'] = self.__getColorElement()
        elif(cmd == "get_camera_offset"):
            outData['CameraOffset'] = self.__getCameraOffset().__dict__
        elif(cmd == "get_gps_offset"):
            outData['GpsOffset'] = self.__getGpsOffset().__dict__
    outData = self.convert_numpy_to_list(outData)
    server.send_message(client, json.dumps(outData))


从程序运行的过程可以看出，Jetson端通过神经网络识别场地物体的坐标位置和距离，数据传输线程将这些数据实时发送给Brian，实时数据需要放置在一个公共空间，为防止同时写入和读出数据的冲突，在主程序开始的 self.v5Web.start()创建了需要互锁的变量：<br>
self.__detections 识别物体的位置距离数据。<br>
self.__colorImage 彩色图像。<br>
self.__depthImage 深度图像。<br>
self.__stats 统计数据。<br>
self.__dataLock 互锁变量。<br>

In [ ]:
    self.__detections = AIRecord(Position(0, 0, 0, 0, 0, 0, 0, 0) [])
    self.__colorImage = None
    self.__depthImage = None 
    self.__stats = Statistics(0, 0, 0, 0, 0, 0, False)
    self.__dataLock = Lock()
    self.__server.run_forever(True)

在主程序run中，每次循环都刷新这些变量。

In [ ]:
    self.set_v5(aiRecord)
    self.rendering.set_images(output, depth_map)
    self.rendering.set_detection_data(aiRecord)
    self.rendering.set_stats(self.stats, self.v5Pos, start_time, invoke_time, run_time)

例如，在run()修改当前视场的彩色和深度图像：

In [ ]:
self.rendering.set_images(output, depth_map)

使用Rendering的方法：<br>

In [ ]:
def set_images(self, output, depth_image):
    # Update web data with color and depth images
    self.web_data.setColorImage(output)
    self.web_data.setDepthImage(depth_image)

调用V5Web的函数，注意这里使用了定义的互锁__dataLock。<br>

In [ ]:
def setColorImage(self, image):
    # Updates the color image data
    self.__dataLock.acquire()
    self.__colorImage = image
    self.__dataLock.release()
    
def setDepthImage(self, image):
    # Updates the depth image data
    self.__dataLock.acquire()
    self.__depthImage = image
    self.__dataLock.release()

这是因为__message_received()在发送图像数据时，也是在这些空间获取数据：<br>

In [ ]:
    elif(cmd == "g_depth"):
        outData['Depth'] = self.__getDepthElement()
    elif(cmd == "g_color"):
        outData['Color'] = self.__getColorElement()

为避免发生同时进行数据刷新和数据取出的冲突，数据存入和取出双方都使用了互锁机制。<br>

In [ ]:
def __getColorElement(self):
        # Returns the color image data encoded in base64
        outData = {}
        imageData = {}
        
        self.__dataLock.acquire()
        pixelData = self.__colorImage
        self.__dataLock.release()

        if(len(pixelData) > 0):
            imageData['Valid'] = True
            imageData['Width'] = pixelData.shape[1]
            imageData['Height'] = pixelData.shape[0]
            imageData['Data'] = base64.b64encode(np.array(cv2.imencode(".jpeg", pixelData)[1]).tobytes()).decode('utf-8')
        else:
            imageData['Valid'] = False
            imageData['Error'] = "Image Unavailable"

        outData['Image'] = imageData

        return outData

V5Web.py中还定义了现场需要监视的统计数据，CPU温度、视频分辨率、工作时间、GPS连接状态等。如class Statistics所示：<br>

In [ ]:
def __init__(self, fps: float, invokeTime: float, cpuTemp: float, videoWidth: int,
                 videoHeight: int, runTime: int, gpsConnected: bool):
    # Initialize statistics attributes such as FPS, CPU temperature, video dimensions, runtime, and GPS connection status
    self.fps = fps
    self.invokeTime = invokeTime
    self.cpuTemp = cpuTemp
    self.videoWidth = videoWidth
    self.videoHeight = videoHeight
    self.runTime = runTime
    self.gpsConnected = gpsConnected

在客户提出数据请求时，向客户端送出。<br>

In [ ]:
elif(cmd == "g_stats"):
    outData['Stats'] = self.__getStatsElement()

准备数据传输包时，也使用了互锁机制。<br>

In [ ]:
    def __getStatsElement(self):
        # Returns the current statistics elements
        outData = {}
        
        self.__dataLock.acquire()
        nowStats = self.__stats
        self.__dataLock.release()

        outData['FPS'] = nowStats.fps
        outData['InferTime'] = nowStats.invokeTime
        outData['VideoWidth'] = nowStats.videoWidth
        outData['VideoHeight'] = nowStats.videoHeight
        outData['RunTime'] = nowStats.runTime
        outData['GPSConnected'] = nowStats.gpsConnected
        outData['CPUTempurature'] = nowStats.cpuTemp

        return outData

VAIC_23_24软件项目定义了客户端数据请求的各个命令，对远程客户端运行机制没有给出任何技术说明，需要参赛者自行进行客户端运行程序的编程。<br>
客户端远程显示画面如图-2所示。<br>

<img src="./实况.png" >

图-2 远程终端上场地的实况和物体识别位置距离数据显示<br>

###    6.&nbsp; &nbsp;Jetson接收Brian端GPS数据程序

V5Position.py的class V5GPS是Jetson接收Brian发送的Vex GPS的功能模块。<br>
在创建V5GPS后，执行start(self)函数启动接收线程：<br>

In [ ]:
    def start(self):
        # Starts the GPS thread
        self.__started = True
        self.__thread = threading.Thread(target=self.__run, args=())
        self.__thread.daemon = True
        self.__thread.start()

线程目标函数def __run(self)如注释所述：用于运行GPS数据的读取和处理、包括建立连接、数据读取、坐标转换以及状态处理、为GPS偏移量设置局部变量。这里仅对数据接收部分进行解释，从Brian传送的GPS数据格式为：<br>
    frameCount = frameCount<br>
    status = status<br>
    x = x<br>
    self.y = y<br>
    self.z = z<br>
    self.azimuth = azimuth<br>
    self.elevation = elevation<br>
    self.rotation = rotation<br>
从下列线程的目标程序的片段中，可以看出，传送数据长度为16字节，数据报头为“xCC，x33”，我们将在下面介绍的Brian 发送Vex GPS数据的C++程序中，发现相同的信息报头和上述相应的数据结构。<br>
线程在读出报头后，对已接收帧计数加一，然后读出两字节状态和相应的三个两字节的位置坐标及三个两字节的偏角数值。<br>

In [ ]:
    while self.__started:
        # Read data from the serial port
        data = self.__ser.read_until(b'\xCC\x33')
        if(len(data) == 16):
            self.__frameCount = self.__frameCount + 1
            status = data[1]
            x, y, z, az, el, rot = struct.unpack('<hhhhhh', data[2:14])

经过坐标的量纲变换和角度的弧度变换，利用azimuth角度计算新的偏距等计算之后，将接收数据存入公共空间self.__position。<br>

In [ ]:
    if(status == 20):
        x, y = self.__filter.update(x, y)
        self.__positionLock.acquire()
        self.__position.x = x
        self.__position.y = y
        self.__position.z = z
        self.__position.azimuth = az
        self.__position.elevation = el
        self.__position.rotation = rot
        self.__position.status = localStatus
        self.__position.frameCount = self.__frameCount
        self.__positionLock.release()

这里需要提及的是程序在此对输入的x、y坐标进行了平滑处理。<br>
.x, y = self.__filter.update(x, y)<br>
V5Position.py在文件开始处导入了filter.py对class LiveFilter的定义，<br>
from filter import LiveFilter<br>
在主程序class MainApp初始化时，创建实例self.v5Pos = V5GPS()。<br>
V5GPS的def __init__(self, port = None)中self.__filter = LiveFilter(10)创建了窗口为10的filter。<br>

from collections import deque<br>
self.x_buffer = deque(maxlen=window_size)<br>
self.y_buffer = deque(maxlen=window_size)<br>

在Python中，deque是collections模块中的一个双端队列（double-ended queue）类。它支持从两端快速添加和删除元素，当deque中的元素数量达到window_size时，再添加新元素会导致队列中最旧的元素被自动删除。<br>
这样在x_buffer、y_buffer保存最新的window_size个坐标数值，由update(self, x, y)是取其平均值，再送入x、y坐标的公共空间。<br>
也就是说，所谓当前平面位置坐标是前十个GPS测量坐标的平均值，数据的平滑化处理，有效地避免了GPS测量误差，消除了数据的激烈变化，使依据此数据控制的机器人能够运动平滑流畅。<br>


In [ ]:
from collections import deque
import numpy as np

class LiveFilter:
    def __init__(self, window_size=5, output_file='filtered_data_simple.txt'):
        self.window_size = window_size
        self.x_buffer = deque(maxlen=window_size)
        self.y_buffer = deque(maxlen=window_size)
        self.output_file = output_file

    def update(self, x, y):
        # Update x and y buffers
        self.x_buffer.append(x)
        self.y_buffer.append(y)

        # Compute the average of the current buffer
        filtered_x = np.mean(self.x_buffer)
        filtered_y = np.mean(self.y_buffer)

        # Write the filtered data to an output file
        with open(self.output_file, 'a') as f:
            f.write(f"{filtered_x}, {filtered_y}\n")

        return filtered_x, filtered_y

self.__position是接收线程和主程序使用此数据的get_v5Pos()写入读出的公共空间，为避免同时写入和读出数据，引发数据读写冲突，在线程写入时，使用了互锁机制。<br>

In [ ]:
    self.__positionLock.acquire()
    self.__position.x = x
    self.__position.y = y
    self.__position.z = z
    self.__position.azimuth = az
    self.__position.elevation = el
    self.__position.rotation = rot
    self.__position.status = localStatus
    self.__position.frameCount = self.__frameCount
    self.__positionLock.release()

In [ ]:
    def getPosition(self):
        # Retrieves the current position
        self.__positionLock.acquire()
        nowPosition = self.__position
        self.__positionLock.release()

###    7.&nbsp; &nbsp;Brian端C++控制程序详细说明

VAIC_23_24Brian端Cpp程序组成已在前面进行简单介绍。请参考“VAIVC软件解释”。<br>

#####   7-1.  Jetson、Brian端通讯数据格式比较<br>

为配合Jetson端通讯功能，Brian端增加了ai_jetson.h中若干数据结构和称之为ai的命名空间，在此空间中定义了实现数据接收的全部属性和方法，也就是所需的常数、变量及功能函数，这些内容在以下的讲解中需要时再加以说明。<br>
在对Brian端Cpp程序进行说明之前，我们首先看看双方进行通讯的数据结构，显然，为了通讯顺利，双方的通讯数据结构应该尽可能相同。<br>
在Jetson端接收Brian的GPS数据为Position，见V5Comm.py的class Position，其属性格式如下：

In [ ]:
        self.frameCount = frameCount
        self.status = status
        self.x = x
        self.y = y
        self.z = z
        self.azimuth = azimuth
        self.elevation = elevation
        self.rotation = rotation

Brian GPS发送数据格式在目录V5Example的ai_jetson.h 中定义如下：<br>

In [ ]:
typedef struct {
	int32_t		framecnt;       
	int32_t		status;         
	float	    x, y, z;                              
	float	    az;            
    float       el;             
    float       rot;           
} POS_RECORD;

由于python语言对变量无需指定数据类型，其实质完全相同。<br>

同样，Jetson发送给Brian的识别数据在python端定义为：<br>

In [ ]:
detect = V5Comm.Detection(
            int(detection.ClassID),
            float(detection.Prob),
            float(depth),
            imageDet,
            mapDet,
  )

其中，

In [ ]:
            imageDet = V5Comm.ImageDetection(
                int(detection.x),
                int(detection.y),
                int(detection.Width),
                int(detection.Height),
            )

而且，

In [ ]:
mapDet = V5Comm.MapDetection(mapPos[0], mapPos[1], mapPos[2])

 在Brian端，接收数据的结构分别定义为：

In [ ]:
typedef struct {
  int32_t		      classID;	        // The class ID of the object.
  float           probability;      // The probability that this object is in this class (1.0 == 100%)
  float           depth;            // The depth of this object in meters from the camera
  IMAGE_DETECTION   screenLocation;   // The screen coordinates of this object
  MAP_DETECTION     mapLocation;      // The field coordinates of this object
} DETECTION_OBJECT;

其中，

In [ ]:
typedef struct {
	int32_t		x, y;			      
	int32_t		width, height;	
} IMAGE_DETECTION;
typedef struct {
  float     x;      
  float     y;      
  float     z;      
} MAP_DETECTION;

 同样，数据格式完全相同。

最后双方交换数据的最终格式为AI_RECORD，<br>
Jetson端在创建AIRecord实例时,定义为class Posion和class Detection的组合，list[Detection]表示是Detection的列表，<br>
pythond的列表可以随时扩充，所以对list[Detection]维度没有必要加以限制。<br>

In [ ]:
    def __init__(self, position: Position, detections: "list[Detection]"):
        self.position = position
        self.detections = detections

Brian端AI_RECORD定义为：<br>

In [ ]:
typedef struct {<br>
    int32_t		    detectionCount;		          
    POS_RECORD	        pos;                           
    DETECTION_OBJECT	detections[MAX_DETECTIONS];    
  
} AI_RECORD;

这里，DETECTION_OBJECT	detections[MAX_DETECTIONS];定义为数组，由于数组大小不能随时扩充，所以用MAX_DETECTIONS加以限制。<br>
比较两者,多了一个识别物体的计数detectionCount，这是由于python语言可以直接用len(detections)得到识别物体的数量，在python语言环境里不需要设定这个计数。而Cpp语言需要在提取识别物体时，预先知道识别物体个数以建立提取循环。<br>
为照顾Cpp语言的这种特点，Jetson端在发送前数据打包时增加了这个计数。<br>
见V5Comm.py class AIRecord的方法：<br>

In [ ]:
    def to_Serial(self):
        # Convert AIRecord properties to serialized binary format
        data = struct.pack('<i', len(self.detections))
        data += self.position.to_Serial()
        for det in self.detections:
            data += det.to_Serial()
        return data

.data = struct.pack('<i' len(self.detections)),检测detections中detect数，以int格式打包到data中。<br>
.data += self.position.to_Serial(),打包position数据到data。<br>
.for det in self.detections:<br>
              data += det.to_Serial()<br>
遍历detections，将所有detect打包至data。<br>

#####   7-2.  Brian端接收Jetson发送数据过程<br>

主程序执行ai::jetson，jetson_comms;创建了jetson的实例，实例初始化启动Brian接收Jetson目标物位置信息的通讯线程。<br>
线程函数为receive_task，高优先级。<br>

In [ ]:
jetson::jetson() {
    state = jetson_state::kStateSyncWait1;
    thread t1 = thread( receive_task, static_cast<void *>(this) );
    t1.setPriority(thread::threadPriorityHigh);
}
jetson::~jetson() {
}

当Brian端需要新的位置数据时，调用 jetson_comms.request_map()函数。Brian向Jetson发送请求新位置数据要求"AA55CC3301“。<br>
前面提到，当Jetson接收到请求，即可发送最新位置数据。Brian端线程receive_task一直处于运行状态，等待接收传输信息包。<br>

In [ ]:
int
jetson::receive_task( void *arg ) {
    if( arg == NULL)
      return(0);
    jetson *instance = static_cast<jetson *>(arg);
    while(1) {
      int rxchar = getchar();
      if( rxchar >= 0 ) {
        instance->total_data_received++;
        while( instance->parse( (uint8_t)rxchar ) )
          this_thread::yield();
      }
    }
}

在while( instance->parse( (uint8_t)rxchar ) )，读取Jetson传来的信息。

parse()函数读取从Jetson送来的位置数据，参看ai_jetson.cpp的jetson::parse( uint8_t data )，开始部分一直在读取和判断“0xAA,0x55,0xCC,0x33”，<br>
以及检查数据完整性检查的校验算法的CRC32码(Cyclic Redundancy Check 32),计算校验成功后，认定传输的数据正确，进入kStateGoodPacket。<br>
这里只截取了线程receive_task的parse()函数中接收AI_RECORD数据的程序段，显然：<br>
.memcpy(&newMap, &payload.bytes[0], MAP_POS_SIZE);是从接收数据包开始位置提取传送的Posion数据。<br>
.memcpy(&newMap.detections, &payload.bytes[MAP_POS_SIZE],sizeof(DETECTION_OBJECT) * newMap.detectionCount)是从接收数据包Posion以后的位置提取各个detection的数据。<br>
.if(newMap.detectionCount > MAX_DETECTIONS)
      newMap.detectionCount = MAX_DETECTIONS;<br>
如果送来数据量大于Cpp端接收数据的预留空间，后边数据被丢弃。<br>
.memcpy( &last_map, &newMap, sizeof(AI_RECORD));从线程内部变量newMap内容复制到主程序全局变量last_map。<br>

In [ ]:
     kStateGoodPacketcase jetson_state:::
        if( payload_type == MAP_PACKET_TYPE ) {
          AI_RECORD newMap;
          // Parse the payload packet into a AI_RECORD
          memset(&newMap, 0, sizeof(newMap));
          memcpy(&newMap, &payload.bytes[0], MAP_POS_SIZE);
          if(newMap.detectionCount > MAX_DETECTIONS)
            newMap.detectionCount = MAX_DETECTIONS;
          memcpy(&newMap.detections, &payload.bytes[MAP_POS_SIZE], 
                       sizeof(DETECTION_OBJECT) * newMap.detectionCount);
          // lock access to last_map and copy data
          maplock.lock();
          memcpy( &last_map, &newMap, sizeof(AI_RECORD));
          maplock.unlock();
        }
        // timestamp this packet
        last_packet_time = timer.system();
        packets++;
        state = jetson_state::kStateSyncWait1;
        break;

VAIC_23_24项目的Brian端给出的是VEX AI竞赛的程序框架，具体竞赛策略和行动规划由参赛者自行编程确定。<br>
参看main.cpp，自主程序分两部分，隔离式auto_Isolation()和交互式auto_Interaction()。<br>
.Competition.autonomous(autonomousMain);启动自主程序。由于设定 firstAutoFlag = true，<br>
先执行程序auto_Isolation。


In [ ]:
bool firstAutoFlag = true;
void autonomousMain(void) {
  if(firstAutoFlag)
    auto_Isolation();
  else 
    auto_Interaction();
  firstAutoFlag = false;
}

auto_Isolation（void）给出了一个简单的竞赛动作的演示：<br>
.GPS.calibrate();启动校准GPS传感器。<br>
.getObject(),进行一系列的程序调用getObject()->findTarget()->get_data()，读出由线程receive_task接收的并复制到last_map里的位置数据。get_data( AI_RECORD *map )如下：<br>

In [ ]:
int32_t
jetson::get_data( AI_RECORD *map ) {
    int32_t length = 0;
    if( map != NULL ) {
        maplock.lock();
        memcpy( map, &last_map, sizeof(AI_RECORD));
        length = last_payload_length;
        maplock.unlock();
    }

接下来就是简单动作，如注释说明，打开手臂，取球，推入球门。<br>
auto_Isolation（void）完成后进入auto_Interaction()。auto_Interaction()是空函数，由竞赛者自行设计编程。<br>

In [ ]:
void auto_Interaction(void) {
  // Functions needed: evaluate which ball detected is target, go to target (x,y), intake ball, dump ball, 
}

Brian端main()主程序中的循环是实际竞赛的过程，由参赛者自行设计竞赛策略和编程实现，这里简单的演示给出了竞赛中需要的三个基本功能：<br>
.从Jetson端取得目标物的位置数据，然后根据竞赛策略设计，自行选取目标，移动目标，完成当前阶段竞赛动作。jetson_comms.get_data( &local_map );<br>
.如果需要向友机发送必要信息，这里选取的发送信息是本机的位置坐标和偏角，也可以根据需要调整发送格式，发送有利于实现竞赛策略的其他信息。link.set_remote_location( local_map.pos.x, local_map.pos.y, local_map.pos.az, local_map.pos.status );<br>
.向Jetson端发出新目标物的位置坐标请求，继续进行下一阶段的竞赛。jetson_comms.request_map();<br>

In [ ]:
  while(1) {

      jetson_comms.get_data( &local_map );
      link.set_remote_location( local_map.pos.x, local_map.pos.y, local_map.pos.az, local_map.pos.status ); 
      jetson_comms.request_map();
      this_thread::sleep_for(loop_time);
  }

这里对数据要求request_map()与Jetson端运行线程之间的联动进行简单对比说明，request_map()函数的Cpp源程序如下：<br>
Brian是以写文件的方式进行通信，Jetson的通信端口为"/dev/serial1"，数据请求命令为"AA55CC3301"。<br>
Brian端发送请求命令，fwrite( msg, 1, strlen(msg), fp );。<br>



In [ ]:
void
jetson::request_map() {
    if( state != jetson_state::kStateSyncWait1 && timer.time() > 250 ) {
      state = jetson_state::kStateSyncWait1;
    }
    if( state == jetson_state::kStateSyncWait1 ) {
      FILE *fp = fopen("/dev/serial1", "w");
      static char msg[] = "AA55CC3301\r\n"; 
      fwrite( msg, 1, strlen(msg), fp );
      fclose(fp);
    }
}

Jetson线程接收到数据请求，发送位置数据。

In [ ]:
while self.__started:  # Continue reading while thread is started
  # Read data from the serial port
  data = self.__ser.readline().decode("utf-8").rstrip()
    if(data == "AA55CC3301"):
      self.__detectionLock.acquire()
      myPacket = V5SerialPacket(self.__MAP_PACKET_TYPE, self.__detections)
      self.__detectionLock.release()
      data = myPacket.to_Serial()
      self.__ser.write(data)  # Write serialized data to the serial port
    


###   8.&nbsp; &nbsp;Vex5机器人Brian端ai_robot_link.cpp程序说明<br>

在Brian端的Cpp程序中，VAIC_23_24增加了在两个竞赛机器人建立实时信息交换机制ai_link。双机之间的通讯与前面介绍的Jetson与Brian之间的通讯不同之处是双机使用无线模式而Jetson与Brian使用的是有线USB接口模式，<br>
为此在ai命名空间中定义了继承vex::serial_link的class robot_link，vex::serial_link包含了Vex的无线通讯功能。详细内容请参考ai_robot_link.h。<br>
VAIC_23_24竞赛要求必须进入auto_Interaction模式，但没有提供任何双机交互的参考示例，让参赛者充分发挥自己的想象力进行竞赛设计。同时提供了足够的基础功能编程示例，供编程人员参考。<br>
在此，作为进行ai_robot_link主要运行机制介绍的预备知识，对定义了的一些变量和函数功能作些简单说明。<br>

#### 一.数据结构

1.数据包报头，如注释所示，包含：同步码，包长度，包类型，循环冗余校验码。

In [ ]:
typedef struct __attribute__((__packed__)) _packet_header {
    uint8_t    sync[2];    // synchronizing bytes, find the start of a valid packet
    uint8_t    length;     // length of map record payload, does not include header
    uint8_t    type;       // type of packet, 
    uint16_t   crc;        // crc32 of payload, this may need to be removed to allow more payload
} packet_header;

2.同步码:作为传输信息包启始位置识别标识的两字节常数。<br>

In [ ]:
 enum class sync_byte {
    kSync1 = 0xA5,
    kSync2 = 0x5A
 };

3.有效载荷，数据包搭载的有效数据。这里简单设定为机器人位置的(X、Y)和指向(角度)。<br>

In [ ]:
typedef struct __attribute__((__packed__)) _packet_1_payload {
    float      loc_x;  
    float      loc_y;
    float      heading;  
} packet_1_payload;

4.通讯传送时使用的数据包结构，packet_1_t = packet_header + packet_1_payload。<br>

In [ ]:
typedef struct __attribute__((__packed__)) _packet_1_t {
    packet_header     header;
    packet_1_payload  payload;
} packet_1_t;

#### 二.功能函数

1.线程，在ai_robot_link中定义了两种通讯线程，数据信息发送与接收线程。<br>
.rx_task()是接收线程的目标函数，负责接收友机发来的友机当前位置信息。<br>
.tx_task()是发送线程的目标函数，负责向友机发送本机的当前位置信息。<br>
具体功能在分析程序运行时在详细说明。<br>

In [ ]:

static int    rx_task( void *arg );
static int    tx_task( void *arg );

2.传输函数，ai_robot_link中给出了几个示范函数，简单传送或接收了未准确定义(x,y,heading)数值，显示在双机交互状态下数据传输的方法，供实际竞赛规划设计时根据交互数据类型进行修改。这3个函数的共同点都是对被线程使用的数据结构<br>
 &nbsp; &nbsp;        packet_1_t    packet_tx_1;<br>
 &nbsp; &nbsp;        packet_1_t    packet_rx_1;<br>
进行存取。所以使用了<br> 
&nbsp; &nbsp;        vex::mutex    rxlock;<br>
&nbsp; &nbsp;        vex::mutex    txlock;<br>
mutex（互斥锁，全称为"mutual exclusion"）是一种在编程中常用的同步方法，用于在多线程或多进程环境中保护对共享资源的访问，以避免数据竞争(data race)和其他并发问题。当一个线程或进程获得了一个互斥锁时，其他尝试获取该锁的线程或进程将被阻塞等待，直到该锁被释放。<br>

In [ ]:
void set_remote_location( float x, float y, float heading, int32_t status );
void get_local_location( float &x, float &y, float &heading, int32_t &status );
void get_remote_location( float &x, float &y, float &heading );

3.信息解析函数：从接收的信息包解析出传送的位置数据。
.robot_link::process( uint8_t data )，在rx_task线程中依次读出报头、类型、校验码；进行校验码计算，无误后读出传送数据。<br>
与读取Jetson信息的线程中的parse()函数一样,程序先获取独占互斥锁，依次读入同步码、校验码，计算校验成功后，认定传输的数据正确，复制传输数据，释放互斥锁。如下程序段所示，复制无线端口数据到程序变量packet_rx_1.payload。<br>
下面是截取上述过程中复制有效数据的的程序段。

In [ ]:
case comms_state::kStateGoodPacket:
    if( payload_type == RL_LOCATION_PACKET ) {
    // lock access and copy data
        rxlock.lock();
        memcpy( &packet_rx_1.payload, &payload.pak_1, sizeof(packet_1_payload));
        rxlock.unlock();
    }

###   9.&nbsp; &nbsp;Vex5机器人Brian端ai_functions.cpp程序说明<br>


ai_functions是机器人小车运动及抓取器动作的控制程序，实现运动均调用Vex 5的运动控制函数，与用游戏手柄操作控制运动和程序预设运动参数不同，控制参数需要根据Jetson传来的目标位置坐标经过实时计算得到，是AI的伺服随动控制模块<br>
在ai_functions部分，核心的函数是DETECTION_OBJECT findTarget(int type)，参数type是目标的分类，JetsonExample/lables.txt给出了三色球的分类定义：<br>
type(GreenTriball)=0<br>
type(RedTriball)=1<br>
type(BlueTriball)=2<br>
主程序main()首先调用getObject()函数，getObject()一开始直接使用findTarget(int type),如果没有发现目标，驱动机器人转动角度，改变机器人视觉摄像机的视场，重新提取目标物数据，发现目标后，定位抓取器，驱动机器人移动到目标物的抓取位置。<br>

In [ ]:
void getObject(){
    DETECTION_OBJECT target = findTarget(0);
    // If no target found, turn and try to find again
    if (target.mapLocation.x == 0 && target.mapLocation.y == 0){
        Drivetrain.turnFor(45, rotationUnits::deg, 50, velocityUnits::pct);
        target = findTarget(0);
    }
    double rot = Arm.position(rotationUnits::deg);
    intake(rot-1200, 1205);
    // Move to the detected target's position
    moveToPosition(target.mapLocation.x*100, target.mapLocation.y*100);
}

findTarget(int type)程序段如下，创建保存接收Jetson数据和目标物位置的空间，利用jetson_comms.get_data(&local_map)复制线程receive_task接受的数据，这些内容在介绍线程是已经说明。然后根据目标物的类型(三色球的颜色)遍历全部识别物体找出合乎条件距离最近的目标。<br>

In [ ]:
DETECTION_OBJECT findTarget(int type){
    DETECTION_OBJECT target;
    static AI_RECORD local_map;
    jetson_comms.get_data(&local_map);
    double lowestDist = 1000000;
    // Iterate through detected objects to find the closest target of the specified type
    for(int i = 0; i < local_map.detectionCount; i++) {
        double distance = distanceTo(local_map.detections[i].mapLocation.x, local_map.detections[i].mapLocation.y);
        if (distance < lowestDist && local_map.detections[i].classID == type) {
            target = local_map.detections[i];
            lowestDist = distance;
        }
    }
    return target;
}

ai_functions.cpp内其他内容，包括使用Vex GPS传感器的运动控制函数均沿用了VEX 5的环境，这里就无需详细介绍。

###   10.&nbsp; &nbsp;Vex5机器人Brian端doshboard.cpp程序说明<br>
doshboard.cpp是在Brian屏幕上显示Jetson和root_link执行现状的 程序。程序设计为一线程，在主程序开始时调用。


In [ ]:
  // start the status update display
  thread t1(dashboardTask);

线程启动两函数，将Brian屏幕划分为两个区域，分别显示Jetson和robot_link的程序执行状况。

In [ ]:
int
dashboardTask() {
  while(true) {
    // status
    dashboardJetson(    0, 0, 280, 240 );
    dashboardVexlink( 279, 0, 201, 240 );
    // draw, at 30Hz
    Brain.Screen.render();
    this_thread::sleep_for(16);
  }
  return 0;
}

显然，dashboardJetson(    0, 0, 280, 240 )显示了从Jetson通信情况及接收的数据包数量，以及识别物体(最多8个)位置、分类和置信度。

In [ ]:
Brain.Screen.printAt( ox + 10, oy += 15, "Packets   %d", jetson_comms.get_packets() );
Brain.Screen.printAt( ox + 10, oy += 15, "Errors    %d", jetson_comms.get_errors() );
Brain.Screen.printAt( ox + 10, oy += 15, "Timeouts  %d", jetson_comms.get_timeouts() );
Brain.Screen.printAt( ox + 10, oy += 15, "data/sec  %d             ", total_data );
Brain.Screen.printAt( ox + 10, oy += 15, "pkts/sec  %d             ", total_packets );
Brain.Screen.printAt( ox + 10, oy += 15, "count     %d", local_map.detectionCount );
oy += 15; // Skip a line

  // once per second, update data rate stats
if( Brain.Timer.system() > update_time ) {
    update_time = Brain.Timer.system() + 1000;
    total_data = jetson_comms.get_total() - last_data;
    total_packets = jetson_comms.get_packets() - last_packets;
    last_data = jetson_comms.get_total();
    last_packets = jetson_comms.get_packets();
  }
  
  Brain.Screen.setFont( mono12 );
  for(int i=0;i<8;i++ ) {
    if( i < local_map.detectionCount ) {
      Brain.Screen.printAt( ox + 10, oy += 12, "map %d: p:%.2f c:%4d X:%.2f Y:%.2f Z:%.1f",i,
                           local_map.detections[i].probability,
                           local_map.detections[i].classID,
                           (local_map.detections[i].mapLocation.x /*/ -25.4*/),  // mm -> inches
                           (local_map.detections[i].mapLocation.y /*/ -25.4*/),  // mm -> inches
                           (local_map.detections[i].mapLocation.z /*/ 25.4*/)); // mm -> inches
    }

dashboardVexlink( 279, 0, 201, 240 )显示了link通信的状况以及两台机器各自场地的坐标位置机朝向(偏角)。与上述程序相似，这里就不赘述。